In [1]:
import sys, os, _pickle as pickle
import tensorflow as tf
import numpy as np
import nltk
from sklearn.metrics import f1_score

data_dir = 'data'
ckpt_dir = 'checkpoint'
word_embd_dir = 'checkpoint/word_embd'
model_dir = 'checkpoint/model3v1'

word_embd_dim = 100
pos_embd_dim = 25
dep_embd_dim = 25

word_vocab_size = 400001
pos_vocab_size = 10
dep_vocab_size = 21

relation_classes = 19

state_size = 100
batch_size = 10
channels = 3

max_len_path = 70
max_num_childs = 50

lambda_l2 = 0.0001
init_learning_rate = 0.001
decay_steps = 2000
decay_rate = 0.96

In [2]:
with tf.name_scope("input"):
    fp_length = tf.placeholder(tf.int32, shape=[batch_size], name="fp_ength")
    fp = tf.placeholder(tf.int32, [2, batch_size, None], name="full_path")
    sp_length = tf.placeholder(tf.int32, shape=[batch_size, 2], name="sp_length")
    sp = tf.placeholder(tf.int32, [batch_size, 2, None], name="shortest_path")
    sp_pos = tf.placeholder(tf.int32, [batch_size, 2, None], name="sp_pos")
    sp_childs = tf.placeholder(tf.int32, [batch_size, 2, None, None], name="sp_childs")
    sp_num_childs = tf.placeholder(tf.int32, [batch_size, 2, None], name="sp_num_childs")
    relation = tf.placeholder(tf.int32, [batch_size], name="relation")
    y_entity = tf.placeholder(tf.int32, [batch_size, None], name="y_enity")

with tf.name_scope("word_embedding"):
    W = tf.Variable(tf.constant(0.0, shape=[word_vocab_size, word_embd_dim]), name="W")
    embedding_placeholder = tf.placeholder(tf.float32,[word_vocab_size, word_embd_dim])
    embedding_init = W.assign(embedding_placeholder)
    embd_fp_word = tf.nn.embedding_lookup(W,fp[0])
    word_embedding_saver = tf.train.Saver({"word_embedding/W": W})

with tf.name_scope("pos_embedding"):
    W = tf.Variable(tf.random_uniform([pos_vocab_size, pos_embd_dim]), name="W")
    embd_fp_pos = tf.nn.embedding_lookup(W, fp[1])
    pos_embedding_saver = tf.train.Saver({"pos_embedding/W": W})

with tf.name_scope("dep_embedding"):
    W = tf.Variable(tf.random_uniform([dep_vocab_size, dep_embd_dim]), name="W")
    embd_sp = tf.nn.embedding_lookup(W, sp)
    dep_embedding_saver = tf.train.Saver({"dep_embedding/W": W})
    
embd_fp = tf.concat([embd_fp_word, embd_fp_pos], axis=2)
embd_fp_rev = tf.reverse(embd_fp, [1])
fp_length_rev = tf.reverse(fp_length, [0])

In [3]:
def lstm_seq_init(channel, embedding_dim, state_size):
    init_const = tf.zeros([1, state_size])
    with tf.variable_scope(channel):
        W_i = tf.get_variable("W_i",shape=[embedding_dim, state_size] ,initializer=tf.contrib.layers.xavier_initializer())
        U_i = tf.get_variable("U_i",shape=[state_size, state_size] ,initializer=tf.contrib.layers.xavier_initializer())
        b_i = tf.get_variable("b_i", initializer=init_const)

        W_f = tf.get_variable("W_f",shape=[embedding_dim, state_size] ,initializer=tf.contrib.layers.xavier_initializer())
        U_f = tf.get_variable("U_f",shape=[state_size, state_size] ,initializer=tf.contrib.layers.xavier_initializer())
        b_f = tf.get_variable("b_f", initializer=init_const)

        W_o = tf.get_variable("W_o",shape=[embedding_dim, state_size] ,initializer=tf.contrib.layers.xavier_initializer())
        U_o = tf.get_variable("U_o",shape=[state_size, state_size] ,initializer=tf.contrib.layers.xavier_initializer())
        b_o = tf.get_variable("b_o", initializer=init_const)

        W_g = tf.get_variable("W_g",shape=[embedding_dim, state_size] ,initializer=tf.contrib.layers.xavier_initializer())
        U_g = tf.get_variable("U_g",shape=[state_size, state_size] ,initializer=tf.contrib.layers.xavier_initializer())
        b_g = tf.get_variable("b_g", initializer=init_const)

In [4]:
lstm_seq_init("lstm_fw", word_embd_dim + pos_embd_dim, state_size)
lstm_seq_init("lstm_bw", word_embd_dim + pos_embd_dim, state_size)

In [5]:
def cond(i, steps, *agrs):
    return i< steps

In [6]:
init_state_seq = tf.zeros([2, 1, state_size])

In [7]:
x = tf.constant(0)

In [8]:
states_series_ = []
states_series_rev = []

In [9]:
def lstm_seq(input_embd, seq_len, scope):
    def body(j, const, steps, input_embd, states_seq, states_series):
        inputs = tf.expand_dims(input_embd[j], [0])
        
        hs = states_seq[0]
        cs = states_seq[1]
        
        hs_ = states_series[0]
        cs_ = states_series[1]
        with tf.variable_scope(scope, reuse=True):
            W_i = tf.get_variable("W_i")
            U_i = tf.get_variable("U_i")
            b_i = tf.get_variable("b_i")

            W_f = tf.get_variable("W_f")
            U_f = tf.get_variable("U_f")
            b_f = tf.get_variable("b_f")

            W_o = tf.get_variable("W_o")
            U_o = tf.get_variable("U_o")
            b_o = tf.get_variable("b_o")

            W_g = tf.get_variable("W_g")
            U_g = tf.get_variable("U_g")
            b_g = tf.get_variable("b_g")
            
            input_gate = tf.sigmoid(tf.matmul(inputs, W_i) + tf.matmul(hs, U_i) + b_i)
            forget_gate = tf.sigmoid(tf.matmul(inputs, W_f) + tf.matmul(hs, U_f) + b_f)
            output_gate = tf.sigmoid(tf.matmul(inputs, W_o) + tf.matmul(hs, U_o) + b_o)
            gt = tf.tanh(tf.matmul(inputs, W_g) + tf.matmul(hs, U_g) + b_g)
            cs = input_gate * gt + forget_gate * cs
            hs = output_gate * tf.tanh(cs)
            
            hs_ = tf.cond(tf.equal(j, const), lambda: hs, lambda: tf.concat([hs_, hs], 0))
            cs_ = tf.cond(tf.equal(j, const), lambda: cs, lambda: tf.concat([cs_, cs], 0))
            
            states_seq = tf.stack([hs, cs], axis=0)
            states_series = tf.stack([hs_, cs_], axis=0)
            
            return j+1, const, steps, input_embd, states_seq, states_series

    _, _, _, _, _, state_series_seq = tf.while_loop(cond, body, 
            [0, 0, seq_len, input_embd, init_state_seq, init_state_seq],
            shape_invariants=[x.get_shape(), x.get_shape(), x.get_shape(), input_embd.get_shape(),
            init_state_seq.get_shape(), 
            tf.TensorShape([2, None, state_size])])
    
    return state_series_seq

In [10]:
for b in range(batch_size):
    seq_len = fp_length[b]
    input_embd = embd_fp[b]   
    states_series_.append(lstm_seq(input_embd, seq_len, "lstm_fw"))
    
    input_embd = embd_fp_rev[b]   
    seq_len = fp_length_rev[b]
    states_series_rev.append(lstm_seq(input_embd, seq_len, "lstm_bw"))


In [11]:
states_series_bw = tf.reverse(states_series_rev, [2])

In [12]:
for i in range(batch_size):
    if(i==0):
        states_series_fw = tf.expand_dims(states_series_[i], [0])
    else:
        states_series_fw = tf.concat([states_series_fw, tf.expand_dims(states_series_[i], [0])], axis=0)

In [13]:
states_series_fw

<tf.Tensor 'concat_9:0' shape=(10, 2, ?, 100) dtype=float32>

In [14]:
hidden_states_seq = tf.concat([states_series_fw[:,0], states_series_bw[0:,1]], 2)

In [15]:
hidden_states_seq

<tf.Tensor 'concat_10:0' shape=(10, ?, 200) dtype=float32>

In [16]:
dep_input_size = state_size * 2 + dep_embd_dim

In [17]:
def lstm_dep_init(channel, dep_input_size, state_size):
    init_const = tf.zeros([1, state_size])

    with tf.variable_scope(channel):
        W_i = tf.get_variable("W_i", shape=[dep_input_size, state_size], initializer=tf.contrib.layers.xavier_initializer())
        U_i = tf.get_variable("U_i", shape=[state_size, state_size], initializer=tf.contrib.layers.xavier_initializer())
        b_i = tf.get_variable("b_i", initializer=init_const)
        U_it = tf.get_variable("U_it", shape=[state_size, state_size], initializer=tf.contrib.layers.xavier_initializer())

        W_f = tf.get_variable("W_f", shape=[dep_input_size, state_size], initializer=tf.contrib.layers.xavier_initializer())
        U_f = tf.get_variable("U_f", shape=[state_size, state_size], initializer=tf.contrib.layers.xavier_initializer())
        b_f = tf.get_variable("b_f", initializer=init_const)
        U_fsp = tf.get_variable("U_fsp", shape=[2, state_size, state_size], initializer=tf.contrib.layers.xavier_initializer())
        U_ffp = tf.get_variable("U_ffp", shape=[max_num_childs, state_size, state_size], initializer=tf.contrib.layers.xavier_initializer())

        W_o = tf.get_variable("W_o", shape=[dep_input_size, state_size], initializer=tf.contrib.layers.xavier_initializer())
        U_o = tf.get_variable("U_o", shape=[state_size, state_size], initializer=tf.contrib.layers.xavier_initializer())
        b_o = tf.get_variable("b_o", initializer=init_const)
        U_ot = tf.get_variable("U_ot", shape=[state_size, state_size], initializer=tf.contrib.layers.xavier_initializer())

        W_u = tf.get_variable("W_u", shape=[dep_input_size, state_size], initializer=tf.contrib.layers.xavier_initializer())
        U_u = tf.get_variable("U_u", shape=[state_size, state_size], initializer=tf.contrib.layers.xavier_initializer())
        b_u = tf.get_variable("b_u", initializer=init_const)
        U_ut = tf.get_variable("U_ut", shape=[state_size, state_size], initializer=tf.contrib.layers.xavier_initializer())

In [18]:
lstm_dep_init("lstm_btup", dep_input_size, state_size)
lstm_dep_init("lstm_tpdn", dep_input_size, state_size)

In [19]:
init_state = tf.zeros([2, 1, 1, state_size])

In [20]:
def lstm_dep(b, p, start, seq_len, input_embd, input_pos, input_childs, input_num_child, states_seq, init_state_dep, scope):  
    
    def loop_over_seq(index, const, steps, input_pos, input_embd, input_childs, input_num_child, states_seq, states_dep, states_series):
        
        inputs = tf.expand_dims(tf.concat([hidden_states_seq[b][input_pos[p][index]], input_embd[p][index]],0),0)
        childs = input_childs[p][index]
        num_child = input_num_child[p][index]    
        num_child_sp = tf.shape(states_dep[0])[0]
        
        with tf.variable_scope(scope, reuse=True):
            W_i = tf.get_variable("W_i")
            U_i = tf.get_variable("U_i")
            b_i = tf.get_variable("b_i")
            U_it = tf.get_variable("U_it")

            W_f = tf.get_variable("W_f")
            U_f = tf.get_variable("U_f")
            b_f = tf.get_variable("b_f")
            U_fsp = tf.get_variable("U_fsp")
            U_ffp = tf.get_variable("U_ffp")
            
            W_o = tf.get_variable("W_o")
            U_o= tf.get_variable("U_o")
            b_o = tf.get_variable("b_o")
            U_ot = tf.get_variable("U_ot")

            W_u = tf.get_variable("W_u")
            U_u = tf.get_variable("U_u")
            b_u = tf.get_variable("b_u")
            U_ut = tf.get_variable("U_ut")    

            it = tf.matmul(inputs, W_i) + b_i + tf.matmul(states_dep[0][0], U_i)
            ft = tf.matmul(inputs, W_f) + b_f + tf.matmul(states_dep[0][0], U_f)
            ot = tf.matmul(inputs, W_o) + b_o + tf.matmul(states_dep[0][0], U_o)
            ut = tf.matmul(inputs, W_u) + b_u + tf.matmul(states_dep[0][0], U_u)

            def matmul(k, steps, it, ft, ot, ut):
                it += tf.matmul(states_dep[0][k], U_i)
                ft += tf.matmul(states_dep[0][k], U_f) 
                ot += tf.matmul(states_dep[0][k], U_o) 
                ut += tf.matmul(states_dep[0][k], U_u) 
                return k+1, steps, it, ft, ot, ut
            
            _, _, it, ft, ot, ut = tf.while_loop(cond, matmul, [1, num_child_sp, it, ft, ot, ut])


            def child_sum(k, steps, out, U):
                out += tf.matmul(states_seq[0][childs[k]], U)
                return k+1, steps, out, U

            _, _, ht_i, _ = tf.while_loop(cond, child_sum, [0, num_child, it, U_it])
            _, _, ht_o, _ = tf.while_loop(cond, child_sum, [0, num_child, ot, U_ot])
            _, _, ht_u, _ = tf.while_loop(cond, child_sum, [0, num_child, ut, U_ut])

            input_gate = tf.sigmoid(ht_i)
            output_gate = tf.sigmoid(ht_o)
            u_input = tf.tanh(ht_u)

            cell_state = input_gate * u_input 

            def cell_state_sp(k, steps, cell_state):
                _, _, f_sp, _ = tf.while_loop(cond, child_sum, [0, num_child, ft, U_fsp[k]])
                cell_state += tf.sigmoid(f_sp) * states_dep[1][k]
                return k+1, steps, cell_state
            
            _, _, cell_state = tf.while_loop(cond, cell_state_sp, [0, num_child_sp, cell_state])

            def cell_states_fp(k, steps, ctl):
                _, _, f_fp, _ = tf.while_loop(cond, child_sum, [i, num_child, ft, U_ffp[k]])
                ctl += tf.sigmoid(f_fp) * states_seq[1][childs[k]]
                return k+1, steps, ctl

            _, _, cds = tf.while_loop(cond, cell_states_fp, [0, num_child, cell_state])
            
            hds = tf.expand_dims(output_gate * tf.tanh(cds), 0)

            cds = tf.expand_dims(cds, 0)
            
            states_dep = tf.stack([hds, cds], axis=0)

            hds_ = tf.cond(tf.equal(i, const), lambda: states_dep[0],
                           lambda: tf.concat([states_series[0], states_dep[0]], 0))
            
            cds_ = tf.cond(tf.equal(i, const), lambda: states_dep[1],
                           lambda: tf.concat([states_series[1], states_dep[1]], 0))

            states_series = tf.stack([hds_, cds_], axis=0)

        return index+1, const, steps, input_pos, input_embd, input_childs, input_num_child, states_seq, states_dep, states_series
    
    _, _, _, _, _, _, _, _, _, states_series_dep = tf.while_loop(cond, loop_over_seq,[start, start, seq_len,
                            input_pos, input_embd, input_childs, input_num_child, states_seq, init_state_dep,
                            init_state], shape_invariants=[x.get_shape(), x.get_shape(), x.get_shape(),
                            input_pos.get_shape(), input_embd.get_shape(), input_childs.get_shape(),
                            input_num_child.get_shape(), states_seq.get_shape(),
                            tf.TensorShape([2, None, 1, state_size]), tf.TensorShape([2, None, 1, state_size])])
    
    return states_series_dep

In [24]:
lca_series_btup = []
dp_series_tpdn = []

for i in range(batch_size):
    
    input_pos = sp_pos[i]
    input_embd = embd_sp[i]
    input_childs = sp_childs[i]
    input_num_child = sp_num_childs[i]
    
    input_pos_rev = tf.reverse(sp_pos[i], [1])
    input_embd_rev = tf.reverse(embd_sp[i], [1])
    input_childs_rev = tf.reverse(sp_childs[i], [1])
    input_num_child_rev = tf.reverse(sp_num_childs[i], [1])
    
    states_seq_fw = tf.expand_dims(states_series_fw[i], 2)
    states_seq_bw = tf.expand_dims(states_series_bw[i], 2)
    
    s1 = lstm_dep(i, 0, 0, sp_length[i][0]-1, input_embd, input_pos, input_childs, input_num_child,
                 states_seq_fw, init_state, "lstm_btup")
    
    lca_btup = tf.cond(sp_length[i][0]>1, lambda: s1[:,sp_length[i][0]-2], lambda: init_state[:,0])

    s2 = lstm_dep(i, 1, 0, sp_length[i][1], input_embd, input_pos, input_childs, input_num_child,
                 states_seq_bw, init_state, "lstm_btup")
    

    lca_btup = tf.cond(sp_length[i][1]>0, lambda: tf.stack([lca_btup,
                      s2[:,sp_length[i][0]+sp_length[i][1]-2]],axis=1), lambda: tf.expand_dims(lca_btup, 1))

    lca_series_btup.append(lstm_dep(i, 0, sp_length[i][0]-1, sp_length[i][0], input_embd, input_pos, input_childs,
                                   input_num_child, states_seq_fw, lca_btup, "lstm_btup")[0,0])
    
    lca_tpdn = lstm_dep(i, 0, 0, 1, input_embd_rev, input_pos_rev, input_childs_rev, input_num_child_rev, 
                        states_seq_bw, init_state, "lstm_tpdn")
    
    dp1 = lstm_dep(i, 0, 1, sp_length[i][0], input_embd_rev, input_pos_rev, input_childs_rev, input_num_child_rev,
                   states_seq_bw, lca_tpdn, "lstm_tpdn")[0,-1]
    
    dp2 = lstm_dep(i, 1, 0, sp_length[i][1], input_embd_rev, input_pos_rev, input_childs_rev, input_num_child_rev,
                   states_seq_bw, lca_tpdn, "lstm_tpdn")[0,-1]
    
    
    dp_series_tpdn.append(tf.concat([dp1, dp2], 1))
    
    

In [25]:
for i in range(batch_size):
    
    temp = tf.concat([lca_series_btup[i], dp_series_tpdn[i]], 1)
    if(i==0):
        dp_series = temp
    else:
        dp_series = tf.concat([dp_series,temp], axis=0)

In [26]:
dp_series

<tf.Tensor 'concat_39:0' shape=(10, 300) dtype=float32>

In [27]:
with tf.name_scope("hidden_layer_seq"):
    W = tf.Variable(tf.truncated_normal([200, 100], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([100]), name="b")
    y_hidden_layer = []
    y_hl = tf.zeros([1, 100])
    for batch in range(batch_size):
        s_seq = tf.expand_dims(hidden_states_seq[batch], 1)
        
        def matmul_hl(j, const, steps, input_seq, out_seq):
            temp = tf.tanh(tf.matmul(input_seq[j], W) + b)
            out_seq = tf.cond(tf.equal(j, const), lambda: temp, lambda: tf.concat([out_seq, temp], 0))
            return j+1, const, steps, input_seq, out_seq
        
        _, _, _, _, output_seq = tf.while_loop(cond, matmul_hl, 
                                [0, 0, fp_length[batch], s_seq, y_hl],
                                shape_invariants=[x.get_shape(), x.get_shape(), x.get_shape(), 
                                s_seq.get_shape(), tf.TensorShape([None, 100])])
        y_hidden_layer.append(output_seq)

In [28]:
with tf.name_scope("dropout_hidden_layer_seq"):
    y_hidden_layer_drop = tf.nn.dropout(y_hidden_layer, 0.3)

In [29]:
with tf.name_scope("softmax_layer_seq"):
    W = tf.Variable(tf.truncated_normal([100, 2], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([2]), name="b")
    
    logits_entity = []
    predictions_entity = []
    lg = tf.zeros([1, 2])
    
    for batch in range(batch_size):
        def matmul_softmax(j, const, steps, y_hl, lg):
            temp = tf.matmul(y_hl[j], W) + b
            lg = tf.cond(tf.equal(j, const), lambda: temp, lambda: tf.concat([lg, temp], 0))
            return j+1, const, steps, y_hl, lg

        y_hl = tf.expand_dims(y_hidden_layer_drop[batch], 1)
        _, _, _, _, logit = tf.while_loop(cond, matmul_softmax, 
                                [0, 0, fp_length[batch], y_hl,lg],
                                shape_invariants=[x.get_shape(), x.get_shape(), x.get_shape(), 
                                y_hl.get_shape(), tf.TensorShape([None, 2])])
        logits_entity.append(logit)
        predictions_entity.append(tf.arg_max(logit, 1))

In [30]:
with tf.name_scope("loss_seq"):
    loss_seq = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits_entity, labels=y_entity))

In [31]:
with tf.name_scope("hidden_layer_dep"):
    W = tf.Variable(tf.truncated_normal([300, 100], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([100]), name="b")
    y_p = tf.tanh(tf.matmul(dp_series, W) + b)

In [32]:
with tf.name_scope("softmax_layer_dep"):
    W = tf.Variable(tf.truncated_normal([100, relation_classes], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([relation_classes]), name="b")
    logits = tf.matmul(y_p, W) + b    

In [33]:
with tf.name_scope("loss_dep"):
    loss_dep = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=relation))

In [34]:
tv_all = tf.trainable_variables()

tv_regu = []

non_reg = ["word_embedding/W:0","pos_embedding/W:0",'dep_embedding/W:0']

for t in tv_all:
    if t.name not in non_reg:
        if t.name.find('b_')==-1:
            if t.name.find('b:')==-1:
                tv_regu.append(t)

In [35]:
with tf.name_scope("total_loss"):
    l2_loss = lambda_l2 * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_regu ])
    total_loss = l2_loss + loss_seq + loss_dep

In [36]:
global_step = tf.Variable(0, trainable=False, name="global_step")

In [ ]:
learning_rate = tf.train.exponential_decay(init_learning_rate, global_step, decay_steps, decay_rate, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(total_loss, global_step=global_step)

In [35]:
f = open('data/vocab.pkl', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"
word2id[unknown_token] = word_vocab_size -1
id2word[word_vocab_size-1] = unknown_token

pos_tags_vocab = []
for line in open('data/pos_tags.txt'):
        pos_tags_vocab.append(line.strip())

dep_vocab = []
for line in open('data/dependency_types.txt'):
    dep_vocab.append(line.strip())

relation_vocab = []
for line in open('data/relation_types.txt'):
    relation_vocab.append(line.strip())

rel2id = dict((w, i) for i,w in enumerate(relation_vocab))
id2rel = dict((i, w) for i,w in enumerate(relation_vocab))

pos_tag2id = dict((w, i) for i,w in enumerate(pos_tags_vocab))
id2pos_tag = dict((i, w) for i,w in enumerate(pos_tags_vocab))

dep2id = dict((w, i) for i,w in enumerate(dep_vocab))
id2dep = dict((i, w) for i,w in enumerate(dep_vocab))

pos_tag2id['OTH'] = 9
id2pos_tag[9] = 'OTH'

dep2id['OTH'] = 20
id2dep[20] = 'OTH'

JJ_pos_tags = ['JJ', 'JJR', 'JJS']
NN_pos_tags = ['NN', 'NNS', 'NNP', 'NNPS']
RB_pos_tags = ['RB', 'RBR', 'RBS']
PRP_pos_tags = ['PRP', 'PRP$']
VB_pos_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
_pos_tags = ['CC', 'CD', 'DT', 'IN']

def pos_tag(x):
    if x in JJ_pos_tags:
        return pos_tag2id['JJ']
    if x in NN_pos_tags:
        return pos_tag2id['NN']
    if x in RB_pos_tags:
        return pos_tag2id['RB']
    if x in PRP_pos_tags:
        return pos_tag2id['PRP']
    if x in VB_pos_tags:
        return pos_tag2id['VB']
    if x in _pos_tags:
        return pos_tag2id[x]
    else:
        return 9

In [36]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

In [ ]:
# f = open('data/word_embedding', 'rb')
# word_embedding = pickle.load(f)
# f.close()
# sess.run(embedding_init, feed_dict={embedding_placeholder:word_embedding})
# word_embedding_saver.save(sess, word_embd_dir + '/word_embd')

In [ ]:
# model = tf.train.latest_checkpoint(model_dir)
# saver.restore(sess, model)

In [ ]:
latest_embd = tf.train.latest_checkpoint(word_embd_dir)
word_embedding_saver.restore(sess, latest_embd)

In [39]:
f = open('data/train_pathsv3', 'rb')
words_seq, deps_seq, pos_tags_seq, word_path1, word_path2, dep_path1, dep_path2, pos_tags_path1, pos_tags_path2, pos_path1, pos_path2, childs_path1, childs_path2 = pickle.load(f)
f.close()

In [75]:
len_path1 = []
len_path2 = []

for w in word_path1:
    len_path1.append(len(w))

for w in word_path2:
    len_path2.append(len(w))

In [78]:
childs_path1[4][0]

[]

In [73]:
word_path2[4]

[]

In [74]:
len_path2[4]

0

In [12]:
relations = []
for line in open('data/train_relations.txt'):
    relations.append(line.strip().split()[1])

length = len(word_p)
num_batches = int(length/batch_size)

for i in range(length):
    for j, word in enumerate(word_p[i]):
        word = word.lower()
        word_p[i][j] = word if word in word2id else unknown_token 
    for l, d in enumerate(dep_p[i]):
        dep_p[i][l] = d if d in dep2id else 'OTH'
        
word_p_ids = np.ones([length, max_len_path],dtype=int)
pos_p_ids = np.ones([length, max_len_path],dtype=int)
dep_p_ids = np.ones([length, max_len_path],dtype=int)
rel_ids = np.array([rel2id[rel] for rel in relations])
path_len = np.array([len(w) for w in word_p], dtype=int)

for i in range(length):
    for j, w in enumerate(word_p[i]):
        word_p_ids[i][j] = word2id[w]
        
    for j, w in enumerate(pos_p[i]):
        pos_p_ids[i][j] = pos_tag(w)
        
    for j, w in enumerate(dep_p[i]):
        dep_p_ids[i][j] = dep2id[w]

In [13]:
num_epochs = 10
for i in range(num_epochs):
    loss_per_epoch = 0
    for j in range(num_batches):
        feed_dict = {
            path_length:path_len[j*batch_size:(j+1)*batch_size],
            word_ids:word_p_ids[j*batch_size:(j+1)*batch_size],
            pos_ids:pos_p_ids[j*batch_size:(j+1)*batch_size],
            dep_ids:dep_p_ids[j*batch_size:(j+1)*batch_size],
            y:rel_ids[j*batch_size:(j+1)*batch_size]}
        _, _loss, step = sess.run([optimizer, total_loss, global_step], feed_dict)
        loss_per_epoch +=_loss
        if (j+1)%num_batches==0:
            print("Epoch:", i+1,"Step:", step, "loss:",loss_per_epoch/num_batches)
    saver.save(sess, model_dir + '/model')
    print("Saved Model")

Epoch: 1 Step: 800 loss: 2.85300489247
Saved Model
Epoch: 2 Step: 1600 loss: 2.73827668965
Saved Model
Epoch: 3 Step: 2400 loss: 2.70001435518
Saved Model
Epoch: 4 Step: 3200 loss: 2.68624746531
Saved Model
Epoch: 5 Step: 4000 loss: 2.68042603165
Saved Model
Epoch: 6 Step: 4800 loss: 2.67750604913
Saved Model
Epoch: 7 Step: 5600 loss: 2.67583220631
Saved Model
Epoch: 8 Step: 6400 loss: 2.67482194766
Saved Model
Epoch: 9 Step: 7200 loss: 2.67411908716
Saved Model
Epoch: 10 Step: 8000 loss: 2.67369878128
Saved Model


In [ ]:
# training accuracy
all_predictions = []
for j in range(num_batches):
     feed_dict = {
            path_length:path_len[j*batch_size:(j+1)*batch_size],
            word_ids:word_p_ids[j*batch_size:(j+1)*batch_size],
            pos_ids:pos_p_ids[j*batch_size:(j+1)*batch_size],
            dep_ids:dep_p_ids[j*batch_size:(j+1)*batch_size],
            y:rel_ids[j*batch_size:(j+1)*batch_size]}
    batch_predictions = sess.run(predictions, feed_dict)
    all_predictions.append(batch_predictions)

y_pred = []
for i in range(num_batches):
    for pred in all_predictions[i]:
        y_pred.append(pred)

count = 0
for i in range(batch_size*num_batches):
    count += y_pred[i]==rel_ids[i]
accuracy = count/(batch_size*num_batches) * 100

print("training accuracy", accuracy)

In [25]:
f = open('data/test_lca_paths', 'rb')
word_p, dep_p, pos_p = pickle.load(f)
f.close()

relations = []
for line in open('data/test_relations.txt'):
    relations.append(line.strip().split()[0])

length = len(word_p1)
num_batches = int(length/batch_size)

for i in range(length):
    for j, word in enumerate(word_p[i]):
        word = word.lower()
        word_p[i][j] = word if word in word2id else unknown_token 
    for l, d in enumerate(dep_p[i]):
        dep_p[i][l] = d if d in dep2id else 'OTH'
        
word_p_ids = np.ones([length, max_len_path],dtype=int)
pos_p_ids = np.ones([length, max_len_path],dtype=int)
dep_p_ids = np.ones([length, max_len_path],dtype=int)
rel_ids = np.array([rel2id[rel] for rel in relations])
path_len = np.array([len(w) for w in word_p], dtype=int)

for i in range(length):
    for j, w in enumerate(word_p[i]):
        word_p_ids[i][j] = word2id[w]
        
    for j, w in enumerate(pos_p[i]):
        pos_p_ids[i][j] = pos_tag(w)
        
    for j, w in enumerate(dep_p[i]):
        dep_p_ids[i][j] = dep2id[w]

# test predictions
all_predictions = []
for j in range(num_batches):
     feed_dict = {
            path_length:path_len[j*batch_size:(j+1)*batch_size],
            word_ids:word_p_ids[j*batch_size:(j+1)*batch_size],
            pos_ids:pos_p_ids[j*batch_size:(j+1)*batch_size],
            dep_ids:dep_p_ids[j*batch_size:(j+1)*batch_size],
            y:rel_ids[j*batch_size:(j+1)*batch_size]}
    batch_predictions = sess.run(predictions, feed_dict)
    all_predictions.append(batch_predictions)

y_pred = []
for i in range(num_batches):
    for pred in all_predictions[i]:
        y_pred.append(pred)

count = 0
for i in range(batch_size*num_batches):
    count += y_pred[i]==rel_ids[i]
accuracy = count/(batch_size*num_batches) * 100

print("test accuracy", accuracy)

ValueError: Cannot feed value of shape (2, 10) for Tensor 'input_1/path1_length:0', which has shape '(10,)'